In [278]:
import requests
import pandas as pd
import os
import re
import math
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from collections import Counter
from natsort import natsorted

In [286]:
episodes = []
title=[]
types = []
popularity = []
description=[]
related=[]
release=[]
animeScore=[]
voices=[]
staff=[]
ranks=[]
# positioning
folder = r"./tsv_anime/"
# iter over the file
for anime in tqdm(natsorted(os.listdir(folder))):
    df = pd.read_csv(folder+anime, sep = "\t")
    episodes.append(df["animeNumEpisode"][0])
    types.append(df["animeType"][0])
    popularity.append(df["animePopularity"][0])
    description.append(df["animeDescription"][0])
    title.append(df["animeTitle"][0])
    related.append(df["animeRelated"][0])
    release.append(df["releaseDate"][0])
    animeScore.append(df["animeScore"][0])
    voices.append(df["animeVoices"][0])
    staff.append(df["animeStaff"][0])
    ranks.append(df["animeRank"][0])
    

100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:03<00:00, 446.61it/s]


In [287]:
df

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
0,Konjiki no Gash Bell!!,TV,150,2003-04-06,2006-03-26,107912,7.54,46265,1499,1519,"Takamine Kiyomaro, a depressed don't-care-abou...","['Konjiki no Gash!!', 'Konjiki no Gash Bell!!:...","['Bell, Gash', 'Takamine, Kiyomaro', 'Folgore,...","['Ootani, IkueJapanese', 'Sakurai, TakahiroJap...","[[['Hamano, Takatoshi', 'Producer'], ['Kaizawa..."


## Change datatypes 

change title[316] datatype to string

In [288]:
import numpy as np
print(type(title[316]))
title[316]=title[316].item()
print(type(title[316]))
title[316]=str(title[316])
print(type(title[316]))

<class 'numpy.int64'>
<class 'int'>
<class 'str'>


In [289]:
for i in range(len(staff)):
    if(type(staff[i])!=str):
        staff[i]=staff[i].item()
        staff[i]=str(staff[i])
    #print(type(staff[i]))

In [290]:
for i in range(len(voices)):
    if(type(voices[i])!=str):
        voices[i]=voices[i].item()
        voices[i]=str(voices[i])
    #print(type(staff[i]))

In [292]:
for i in range(len(ranks)):
    ranks[i]=ranks[i].item()

### We used cosine to determine  how similar the documents are irrespective of their size is to the entered query.Does the text match each other? : (Description,AnimeStaff,Voices)

In [156]:
WORD = re.compile(r"\w+")

In [116]:
def cosine_text(input_text,comparison_text):
    intersection = set(input_text.keys()) & set(comparison_text.keys())
    numerator = sum([input_text[x] * comparison_text[x] for x in intersection])
    sum1 = sum([input_text[x] ** 2 for x in list(input_text.keys())])
    sum2 = sum([input_text[x] ** 2 for x in list(comparison_text.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator  

In [117]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [118]:
def cosine(input_text,comparison_text):
    vector1 = text_to_vector(input_text)
    vector2 = text_to_vector(comparison_text)
    cosine = cosine_text(vector1, vector2)
    return(cosine)

### How to text similar:Type

In [205]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

### For num_episode variable, we have classified the amount of anime into 4 categories:
##### 0 is film
##### 1 is short
##### 2 is medium
##### 3 is large


First we define the category of the input data

In [119]:
def identify_category(input_eps,next_eps):
    if(type(next_eps) == str):
        return 0
    score=0
    if(input_eps==1):
        category=0
    elif(input_eps<25 and input_eps>=8):
        category=1
    elif(input_eps<65 and input_eps>=25):
        category=2
    else:
        category=3
        
    if(next_eps==1):
        category1=0
    elif(next_eps<25 and input_eps>=8):
        category1=1
    elif(next_eps<65 and input_eps>=25):
        category1=2
    else:
        category1=3
    return(category,category1)

Calculated their differences. Depending on the number of episodes, we will be able to determine the greatest suitable anime by length

In [120]:
def episode_score(input_eps,next_eps):
    if(type(next_eps)==str):
        return 0
    category=(identify_category(input_eps,next_eps))
    #print(category)
    if(category[0]==category[1]):
        return 1
    elif(abs(category[0]-category[1])==1):
        return 0.66
    elif(abs(category[0]-category[1])>1):
        return 0.33
    else:
        return 0

## Rank

The logic of ranking metric is that we push away from the request, if the query anime was given to us by the top 100 anime, we of course are looking for a suitable one from the top 100 or from the top 1000

In [298]:
def ranking(input_rank,next_rank):
    diff=abs(input_rank-next_rank)
    if(diff<=100):
        return 1
    elif(diff>100 and diff<=1000):
        return 0.66
    else:
        return 0.33

### Date

We created a date metric to determine how late and newer the anime is with the query being compared.

In [207]:
def date(input_date,releaseDate):
    diff = (abs(datetime.strptime(input_date,'%Y-%m-%d')-datetime.strptime(releaseDate,'%Y-%m-%d')))
    days = diff.days
    #Anime came out in the same season
    if(days>0 and days<=100):
        return 1
    #Differences in 1 year
    elif(days>100 and days<=300):
        return 0.5
    return(0)

print(date(date_query,release[0]))

0.5


## Popularity metric

The logic of this metric is similar to the rank metric.

In [330]:
def score_pop(input_pop,next_pop):
    diff=abs(input_pop-next_pop)
    if(input_pop<=100 and diff<=100):
        return 0.66
    elif(input_pop>100 and pop<=1000 and diff>100):
        return 0.44
    else:
        return 0.22

## animeScore metric

We also categorized the anime by a rating, like metric number episodes
#### 1 High  
#### 2 Average 
#### 3 Low

In [337]:
def identify_s(input_s,next_s):
    if(input_s>=8):
        category=1
    elif(input_s<8 and input_s>=5):
        category=2
    else:
        category=3
       
    if(next_s>=8):
        category1=1
    elif(next_s<8 and next_s>=5):
        category1=2
    else:
        category1=3
    return(category,category1)

In [341]:
def score_s(input_s,next_s):
    category = identify_s(input_s,next_s)
    #print(category)
    if(category[0]==category[1]):
        return 1
    elif(abs(category[0]-category[1])==1):
        return 0.66
    elif(abs(category[0]-category[1])>1):
        return 0.33
    else:
        return 0

In [316]:
#for i in range(len(popularity)):
    #print(popularity[i])

### Final score metric

In [349]:
def final_score1(i,input_type,next_type,input_eps,next_eps,input_title,next_title,input_related,next_related,input_description,next_description,input_date,next_date,input_voice,next_voice,input_staff,next_staff):
    #Type variable
    #print(title[i])
    score1=similar(input_type,next_type)
    #print("Type_score",score1)
    #Episodes variable
    score2=episode_score(input_eps,next_eps)
    #print("Episode_score",score2)
    #Title variable
    score3=cosine(input_title,next_title)
    #print("Title_score",score3)
    #Related anime variable
    score4=cosine(input_related,next_related)
    #print("Related_score",score4)
    #Description variable
    score5=cosine(input_description,next_description)
    #print("Description_score",score5)
    #Season date variable
    score6=date(input_date,next_date)
    #print("Season_score",score6)
    #Voices variable
    score7=cosine(input_voice,next_voice)
    #Staff  variable
    score8=cosine(input_staff,next_staff)
    return(score1+score2+score3+score4+score5+score6+score7+score8)

In [331]:
def final_score2(input_rank,next_rank,input_pop,next_pop,input_s,next_s):
    #Rank
    score9 = ranking(input_rank,next_rank)
    #Popularity
    score10=score_pop(input_pop,next_pop)
    #animeScore
    score11=score_s(input_s,next_s)
    return(score9+score10+score11)


In [143]:
with open("./anime_url.txt", "r", encoding = "utf-8") as f:
    urls = f.readlines()

In [350]:
title_query="Death Note"
type_query="TV"
ep_query=36
related_query="Death Note: Rewrite"
description_query="A shinigami, as a god of death, can kill any person—provided they see their victim's face and write their victim's name in a notebook called a Death Note. One day, Ryuk, bored by the shinigami lifestyle and interested in seeing how a human would use a Death Note, drops one into the human realm.High school student and prodigy Light Yagami stumbles upon the Death Note and—since he deplores the state of the world—tests the deadly notebook by writing a criminal's name in it. When the criminal dies immediately following his experiment with the Death Note, Light is greatly surprised and quickly recognizes how devastating the power that has fallen into his hands could be.With this divine capability, Light decides to extinguish all criminals in order to build a new world where crime does not exist and people worship him as a god. Police, however, quickly discover that a serial killer is targeting criminals and, consequently, try to apprehend the culprit. To do this, the Japanese investigators count on the assistance of the best detective in the world: a young and eccentric man known only by the name of L."
date_query="2006-08-07"
voices_query="Yamaguchi, Kappei,Miyano, Mamoru,Nakamura, Shidou"
staff_query="Maruyama, Masao Producer Tamura, Manabu Produceru Araki, Tetsurou Director, Episode Director, Storyboard, Theme Song LyricsYamada, ChiakiSound Director"
rank_query=66
pop_query=1
score_query=9.1
scores=[]
for i in range(len(episodes)):
    score1=final_score1(i,type_query,types[i],ep_query,episodes[i],title_query,title[i],related_query,related[i],description_query,description[i],date_query,release[0],voices_query,voices[i],staff_query,staff[i])
    score2=final_score2(rank_query,ranks[i],pop_query,popularity[i],score_query,animeScore[i])
    score=score1+score2
    scores.append([score,i])
sorted_scores=sorted(scores, reverse=True)


df_score=[]
df_title=[]
df_urls=[]
for i in range(len(episodes)):
    df_score.append(sorted_scores[i][0])
    df_title.append(title[sorted_scores[i][1]])
    df_urls.append(urls[sorted_scores[i][1]])
df1 = pd.DataFrame (df_score, columns = ['Score:'])
df2 = pd.DataFrame (df_title, columns = ['Title:'])
df3 = pd.DataFrame (df_urls, columns = ['URL:'])
pdList = [df1, df2, df3 ]  
new_df = pd.concat(pdList, axis=1)
display(new_df)

,Score:,Title:,URL:
0,10.017335,Death Note,https://myanimelist.net/anime/1535/Death_Note\n
1,6.890816,Death Parade,https://myanimelist.net/anime/28223/Death_Para...
2,6.825852,Aoi Bungaku Series,https://myanimelist.net/anime/7193/Aoi_Bungaku...
3,6.718537,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...
4,6.704780,Kimetsu no Yaiba,https://myanimelist.net/anime/38000/Kimetsu_no...
...,...,...,...
1495,1.590943,Detective Conan: Conan vs. Kid - Shark & Jewel,https://myanimelist.net/anime/9785/Detective_C...
1496,1.589411,Afro Samurai Movie,https://myanimelist.net/anime/13709/Afro_Samur...
1497,1.589411,Date A Bullet: Nightmare or Queen,https://myanimelist.net/anime/42423/Date_A_Bul...
1498,1.578433,Yuuseiboushi,https://myanimelist.net/anime/49838/Yuuseibous...
